# Investigation of alternative to baseline-dependent averaging.

The idea is to partition the visibility into inner and outer parts. The density of samples in the inner parts of the uv plane is much greater so we can afford to use coarser gridding but with the same field of view as used for the
further out data. The computation required for the inner, dense region of the Fourier plane is just a complex addition, instead of a complete gridding kernel complex add. This approach should rival two-region baseline-dependent averaging.

In this notebook, we investigate the consequences of imaging in this way.

In [ ]:
%matplotlib inline

import os
import sys

from time import clock

sys.path.append(os.path.join('..', '..'))

from matplotlib import pylab

pylab.rcParams['agg.path.chunksize'] = 10000
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import constants as const
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt
from matplotlib.pyplot import cm 

from arl.visibility.operations import create_visibility
from arl.skymodel.operations import create_skycomponent
from arl.image.operations import show_image, export_image_to_fits, qa_image, reproject_image
from arl.image.iterators import raster_iter
from arl.visibility.iterators import vis_timeslice_iter
from arl.util.testing_support import create_named_configuration
from arl.fourier_transforms.ftprocessor import *

import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

doplot = True

Set up imaging parameters

In [ ]:
params = {'npol': 1,
          'cellsize': 0.001,
          'spectral_mode': 'channel',
          'channelwidth': 5e7,
          'reffrequency': 1e8,
          'kernel':'calculated'}

Construct the SKA1-LOW configuration

In [ ]:
low = create_named_configuration('LOWBD2')

We create the visibility holding the vis, uvw, time, antenna1, antenna2, weight columns in a table. The actual visibility values are zero.

In [ ]:
times = numpy.arange(- numpy.pi / 4.0, + numpy.pi * 1.001 / 4.0, numpy.pi / 160)
frequency = numpy.array([1e8])

reffrequency = numpy.max(frequency)
phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-45.0 * u.deg, frame='icrs', equinox=2000.0)
vt = create_visibility(low, times, frequency, weight=1.0, phasecentre=phasecentre, params=params)

In [ ]:
params = {'npixel': 8192,
          'npol': 1,
          'cellsize': 0.00001,
          'spectral_mode': 'channel',
          'channelwidth': 5e7,
          'reffrequency': 1e8}

model = create_image_from_visibility(vt, params=params)

Now calculate the density of samples in the Fourier plane

In [ ]:
vts, density, densitygrid = weight_visibility(vt, model, params)
print(density.shape)
plt.clf()
uvdist = numpy.sqrt(vts.u*vts.u+vts.v*vts.v)
plt.plot(uvdist, density[:,0,0], '.')
plt.xlabel('UV distance (m)')
plt.ylabel('Density (weight per cell)')
plt.show()

In [ ]:
plt.clf()
plt.imshow(numpy.log(1e-3+densitygrid[0,0,...]))
plt.show()

In [ ]:
def invert_twopart(vis, im, dopsf=False, params=None):
    """Divide vis into inner and outer, image separately, and add
    
    Cut down on processing by gridding the inner (dense) part of Fourier plane with box gridding, 
    and the outer (sparse) part by more accurate PSWF gridding.
    """
    boundary = get_parameter(params, "inner", 0.25)
    
    uvmax = numpy.max(numpy.abs([vis.u, vis.v]))

    inner_rows = (numpy.abs(vis.u) < boundary * uvmax) & (numpy.abs(vis.v) < boundary * uvmax)
    outer_rows = (numpy.abs(vis.u) >= boundary * uvmax) & (numpy.abs(vis.v) >= boundary * uvmax)

    inner_vis = vis.select_rows(inner_rows)
    outer_vis = vis.select_rows(outer_rows)
    

    inner_nvis, outer_nvis = numpy.sum(inner_rows), numpy.sum(outer_rows)
    log.debug("Split in inner (%d rows) and outer (%d rows)" % (inner_nvis, outer_nvis))

    inner_params = params.copy()
    inner_params["cellsize"] = params["cellsize"] / boundary
    inner_params["npixel"] = int(round(params["npixel"] * boundary))
    inner_params["padding"] = int(round(get_parameter(params, "padding", 2) / boundary))

    inner_params["kernel"] = "box"    
    inner_im = create_image_from_visibility(vis, inner_params)
    inner_result, inner_sumwt = invert_2d(inner_vis, inner_im, dopsf, inner_params)

    outer_params = params.copy()
    outer_im = create_image_from_visibility(vis, outer_params)

    outer_result, outer_sumwt = invert_2d(outer_vis, outer_im, dopsf, outer_params)
    
    result, footprint = reproject_image(inner_result, outer_result.wcs, outer_result.data.shape)
    
    result_sumwt = inner_sumwt + outer_sumwt
    result.data = (inner_sumwt * result.data + outer_sumwt * outer_result.data) / result_sumwt
    
    return result, result_sumwt 

Set imaging parameters

In [ ]:
params = {'npixel': 8192,
          'npol': 1,
          'cellsize': 0.00001}

First use invert_2d to get the benchmark time.

In [ ]:
ts = clock()
full_psf, full_sumwt = invert_2d(vt, model, dopsf=True, params=params)
full_psf.data /= full_sumwt
qa_full = qa_image(full_psf)
criterion = 'medianabs'
t_full = clock() - ts
log.debug("Invert_2d took %.1f seconds" % (t_full))


In [ ]:
inners = [1.0/4.0, 1.0/8.0, 1.0/16.0, 1.0/32.0]
times = []
for inner in inners:
    params['inner'] = inner
    ts = clock()
    twopart_psf, twopart_sumwt = invert_twopart(vt, model, dopsf=True, params=params)
    twopart_psf.data /= twopart_sumwt
    qa_twopart = qa_image(full_psf)
    criterion = 'medianabs'
    t_twopart = clock() - ts
    log.debug("Invert_twopart using inner=%.3f took %.1f seconds" % (inner, t_twopart))
    times.append(t_twopart)

In [ ]:
plt.clf()
plt.plot(inners, times, label='twopart', color='g')
plt.axhline(t_full, label='2d', color='r')
plt.xlabel('Fraction of uvmax')
plt.ylabel('Time (s)')
plt.legend()
plt.show()